In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn import datasets
from models.ml_models.ml_models import LinRegModel
import pandas as pd

In [9]:
from catboost import CatBoostRegressor

In [28]:
# a = CatBoostRegressor().fit(X, y)

In [29]:
# sorted(list(a.get_all_params().keys()))

# датасет

In [30]:
X, y = datasets.load_diabetes(return_X_y=True)

pd_X = pd.DataFrame(X)

# тестим model manager

In [31]:
# !pwd

storage_dir = './saved_models'

from models.model_manager import ModelManager
mm = ModelManager(storage_dir)

In [32]:
mm.list_models()

[]

In [33]:
# mm.model_classes['lin_reg'].get_param_names()

# REST

In [41]:
import requests
import os
import json

In [42]:
url = 'http://127.0.0.1:8080'

# TestClient

In [43]:
from fastapi.testclient import TestClient  
from server.rest.app import app  # Импортируем наш основной файл с FastAPI  
  
# Создаем клиент для тестирования  
client = TestClient(app)  
  

response = client.get("/status/")
response

<Response [200 OK]>

In [44]:
response.content

b'{"status":"online"}'

# На локальном сервере

## models

In [45]:
response = requests.get(os.path.join(url, 'models'))

response

<Response [200]>

In [46]:
response.content

b'{"lin_reg":["copy_X","fit_intercept","n_jobs","positive"],"catboost_reg":["learning_rate","iterations","depth","l2_leaf_reg","border_count","random_strength","bagging_temperature","oddpriority_threshold","feature_border_type","task_type"]}'

## train

In [50]:
pd_X.columns = [str(i) for i in pd_X.columns]
X_dict = pd_X.to_dict(orient='records')
y_list = list(y)

model_spec = {  
    "type": "catboost_reg",  
    "parameters": {  
        # "fit_intercept": True,  
    }  
}

In [52]:
response = requests.post(
    os.path.join(url, 'train'),
    json={ 
        "model_spec": model_spec,
        "features": X_dict,  
        "targets": y_list,
    }
)

response

<Response [200]>

In [53]:
model_id = json.loads(response.content).get('model_id')
model_id

'catboost_reg_e3b0c44298fc1c14_d61ba0368f3c69f5'

## trained_models

In [60]:
response = requests.get(
    os.path.join(url, 'trained_models'),
)

response

<Response [200]>

In [61]:
response.content

b'{"trained_models":["catboost_reg_e3b0c44298fc1c14_d61ba0368f3c69f5"]}'

## predict

In [56]:
response = requests.post(
    os.path.join(url, 'predict'),
    json={ 
        "model_id": model_id,
        "features": X_dict,
    }
)

response

<Response [200]>

In [57]:
len(json.loads(response.content)['predictions'])

442

In [62]:
response = requests.delete(os.path.join(url, 'models', 'catboost_reg_e3b0c44298fc1c14_d61ba0368f3c69f5'))

response

<Response [200]>

In [63]:
response.content

b'{"status":"success","detail":"Model deleted successfully"}'

In [64]:
requests.get(
    os.path.join(url, 'trained_models'),
).content.decode()

'{"trained_models":[]}'

# Общие комменты: